In [2]:
!git clone https://github.com/Mlawrence95/LinkedIn-Tech-Job-Data.git

Cloning into 'LinkedIn-Tech-Job-Data'...


In [2]:
%cd LinkedIn-Tech-Job-Data

J:\Downloads\NLP\kred\LinkedIn-Tech-Job-Data


In [1]:
!pip install -U pip setuptools wheel
!pip install spacy-transformers
#!pip install pytextrank
!python -m spacy download en_core_web_trf
!python -m spacy download en_core_web_md
!python -m spacy download en_core_web_sm


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 2.1/2.1 MB 18.6 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 23.3 MB/s eta 0:00:00


ERROR: To modify pip, please run the following command:
J:\Downloads\NLP\PyEnv\TorchEnv\Scripts\python.exe -m pip install -U pip setuptools wheel

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     -------------------------------------- 460.3/460.3 MB 7.4 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     --------------------------------------- 42.8/42.8 MB 10.6 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_md')



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     --------------------------------------- 12.8/12.8 MB 14.9 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%ls

 Il volume nell'unit… J Š HDD
 Numero di serie del volume: 7C24-9F9D

 Directory di J:\Downloads\NLP\kred

31/01/2023  13:08    <DIR>          .
14/01/2023  11:41    <DIR>          ..
05/01/2023  10:26               107 .gitignore
31/01/2023  13:09    <DIR>          .idea
05/01/2023  00:25    <DIR>          .vscode
05/01/2023  00:25                 0 __init__.py
31/01/2023  11:04    <DIR>          __pycache__
10/01/2023  13:16    <DIR>          apicache-py3
05/01/2023  08:56    <DIR>          base
05/01/2023  11:33         3.655.339 checkpoint.pt
05/01/2023  10:55             1.481 config.yaml
08/01/2023  16:31    <DIR>          core
04/01/2023  17:14       131.711.608 data_dict_1672848868384.pkl
04/01/2023  17:52    <DIR>          datasets
05/01/2023  10:22        10.320.431 first100pairs_test_data.csv
05/01/2023  00:25           175.182 framework.PNG
05/01/2023  00:25            31.065 kred_example.ipynb
31/01/2023  13:08            42.732 KRED_train.ipynb
31/01/2023  11:13         1

In [5]:
import pandas as pd
import csv
from utils import extract_wikidata

In [1]:
df = pd.read_csv('datasets/SO-Indeed/jobs.csv')
records = df.to_dict(orient='records')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\39333\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\39333\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\39333\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\39333\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [32]:

jobs_info = [ 'post_id', 'industries','job_function', 'title', 'abstract', 'post_url', 'entity_info_title', 'entity_info_abstract']

#new_dict = []
djob = {}
#wikidata_ids = []
with open('train_jobs.csv', 'a', encoding='utf-8', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = jobs_info)
    #writer.writeheader()
    for job in records[7713:]:
        djob["post_id"] = job['post_id']
        djob["industries"] = job['Industries']
        djob["job_function"] = job['Job function']
        djob["title"] = job['title']
        djob["abstract"] = extract_wikidata.extract_significant_sentences(job['description']) ##
        djob["post_url"] = job['post_url']

        #doc_trf = nlp_trf(job['description'])
        djob["entity_info_title"] = extract_wikidata.get_extract(job['description'], wikidata_ids)
        djob["entity_info_abstract"] = [] #remember to remove from util.py in for entity in eval(entity_info_abstract)
        #print(djob)
        writer.writerow(djob)


In [7]:
from utils.extract_wikidata import get_ids

wikidata_ids = []
with open('train_jobs.csv', 'r', encoding='utf-8') as csv_file:
    df = pd.read_csv(csv_file)
    entity_info_title_col = df.entity_info_title

get_ids(entity_info_title_col, wikidata_ids)

In [11]:
len(wikidata_ids)

13398

In [10]:
file = open('wikidata_ids.txt', 'a', encoding='utf-8')
for wikidata in wikidata_ids:
	file.write(wikidata + "\n")
file.close()

#news | jobs
#id : post_id
#category: Industries
#subcategory : job function
#title: title
#abstract : requirements
#url : url
#entity_info_title : only one [{label, type, wikidataId, confidence, occurrence offsets, surface forms}]
#entity_info_abstract: all the first 3[{label, type, wikidataId, confidence, occurrence offsets, surface forms}]


data:
  train_news : "./datasets/mind_train_demo/news.tsv" ###
  train_behavior : "./datasets/mind_train_demo/behaviors.tsv"
  valid_news : "./datasets/mind_val_demo/news.tsv"
  valid_behavior : "./datasets/mind_val_demo/behaviors.tsv"
  knowledge_graph : "./datasets/mind_utils_demo/triple2id.txt" --- h t r
  entity_embedding : "./datasets/mind_utils_demo/entity2vecd100.vec" ###
  num_entity_embedding: 3275150
  relation_embedding : "./datasets/mind_utils_demo/relation2vecd100.vec" ---
  entity_index : "./datasets/mind_utils_demo/entity2id.txt" ###
  relation_index : "./datasets/mind_utils_demo/relation2id.txt" ---
  sparse_adj_entity : "./datasets/mind_utils_demo/entity_adj.npy"
  sparse_adj_relation : "./datasets/mind_utils_demo/relation_adj.npy"
  mind_data : "./data_dict_1672848868384.pkl"